# Lab 5: Bias–Variance Tradeoff using the Air Quality Dataset

**Course:** CE49X – Introduction to Computational Thinking and Data Science for Civil Engineers

**Instructor:** Dr. Eyuphan Koç

**Semester:** Fall 2025

---

## Objectives

In this lab, we will:
- Understand the **bias–variance tradeoff** in machine learning
- Implement and compare **linear** and **polynomial regression** models
- Visualize **training** and **testing errors** as model complexity changes
- Interpret **underfitting** and **overfitting** phenomena using real environmental data

## Step 1: Import Required Libraries

In [2]:
# Data manipulation and numerical computation
import pandas as pd
import numpy as np

# Machine learning tools
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Settings for better plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Ignore warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

ModuleNotFoundError: No module named 'sklearn'

## Step 2: Load and Explore the Dataset

The dataset contains hourly responses from a gas multisensor device deployed in an Italian city. We'll use it to predict CO concentration from meteorological variables.

In [9]:
# Load the dataset
# Note: The file uses semicolon as delimiter and comma as decimal separator
df = pd.read_csv('dataset/AirQualityUCI.csv', sep=';', decimal=',')

# Display basic information
print("Dataset shape:", df.shape)
print("\nFirst few rows:")
df.head()

Dataset shape: (9471, 17)

First few rows:


,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,NaN,NaN
1,10/03/2004,19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN
2,10/03/2004,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN
3,10/03/2004,21.00.00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,NaN,NaN
4,10/03/2004,22.00.00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,NaN,NaN


In [ ]:
# Check column names and data types
print("Column names:")
print(df.columns.tolist())
print("\nData types:")
print(df.dtypes)

In [ ]:
# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())
print("\nBasic statistics:")
df.describe()

## Step 3: Data Preprocessing

We need to:
1. Handle missing values (marked as -200)
2. Select relevant features: T (Temperature), RH (Relative Humidity), AH (Absolute Humidity)
3. Select target variable: CO(GT) (True CO concentration)

In [ ]:
# Replace -200 (missing value indicator) with NaN
df_clean = df.replace(-200.0, np.nan)

# Select features and target
features = ['T', 'RH', 'AH']
target = 'CO(GT)'

# Create a clean dataset with only the columns we need
data = df_clean[features + [target]].copy()

# Drop rows with any missing values
data_cleaned = data.dropna()

print(f"Original dataset size: {len(df)} rows")
print(f"After removing missing values: {len(data_cleaned)} rows")
print(f"\nFeatures: {features}")
print(f"Target: {target}")

# Display the cleaned data
data_cleaned.head()

In [ ]:
# Visualize the relationships between features and target
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, feature in enumerate(features):
    axes[idx].scatter(data_cleaned[feature], data_cleaned[target], alpha=0.3, s=10)
    axes[idx].set_xlabel(feature, fontsize=12)
    axes[idx].set_ylabel('CO(GT) [mg/m³]', fontsize=12)
    axes[idx].set_title(f'{feature} vs CO(GT)', fontsize=13, fontweight='bold')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("These scatter plots show the relationship between each feature and the CO concentration.")

## Step 4: Split Data into Training and Testing Sets

We'll use 70% of the data for training and 30% for testing.

In [ ]:
# Separate features and target
X = data_cleaned[features]
y = data_cleaned[target]

# Split into training and testing sets (70-30 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print(f"Training set size: {len(X_train)} samples ({len(X_train)/len(X):.1%})")
print(f"Testing set size: {len(X_test)} samples ({len(X_test)/len(X):.1%})")
print(f"\nFeature matrix shape (training): {X_train.shape}")
print(f"Target vector shape (training): {y_train.shape}")

## Step 5: Train Polynomial Regression Models of Increasing Complexity

We'll train models with polynomial degrees from 1 to 10:
- **Degree 1**: Linear regression (simplest model)
- **Degree 2-4**: Moderate complexity
- **Degree 5-10**: High complexity

For each model, we'll:
1. Transform features using `PolynomialFeatures`
2. Train a `LinearRegression` model
3. Calculate training and testing errors (MSE and RMSE)

In [11]:
# Store results
degrees = range(1, 11)  # Polynomial degrees 1 to 10
train_mse = []
test_mse = []
train_rmse = []
test_rmse = []
train_r2 = []
test_r2 = []

# Train models for each polynomial degree
print("Training polynomial regression models...\n")
print(f"{'Degree':<8} {'Train MSE':<12} {'Test MSE':<12} {'Train RMSE':<12} {'Test RMSE':<12}")
print("-" * 64)

for degree in degrees:
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)
    
    # Train linear regression on polynomial features
    model = LinearRegression()
    model.fit(X_train_poly, y_train)
    
    # Make predictions
    y_train_pred = model.predict(X_train_poly)
    y_test_pred = model.predict(X_test_poly)
    
    # Calculate errors
    train_mse_val = mean_squared_error(y_train, y_train_pred)
    test_mse_val = mean_squared_error(y_test, y_test_pred)
    train_rmse_val = np.sqrt(train_mse_val)
    test_rmse_val = np.sqrt(test_mse_val)
    
    # Calculate R² scores
    train_r2_val = r2_score(y_train, y_train_pred)
    test_r2_val = r2_score(y_test, y_test_pred)
    
    # Store results
    train_mse.append(train_mse_val)
    test_mse.append(test_mse_val)
    train_rmse.append(train_rmse_val)
    test_rmse.append(test_rmse_val)
    train_r2.append(train_r2_val)
    test_r2.append(test_r2_val)
    
    # Print results
    print(f"{degree:<8} {train_mse_val:<12.4f} {test_mse_val:<12.4f} {train_rmse_val:<12.4f} {test_rmse_val:<12.4f}")

print("\nTraining complete!")

Training polynomial regression models...

Degree   Train MSE    Test MSE     Train RMSE   Test RMSE   
----------------------------------------------------------------


NameError: name 'PolynomialFeatures' is not defined

## Step 6: Visualize the Bias-Variance Tradeoff

We'll create a validation curve showing how training and testing errors change with model complexity.

In [ ]:
# Create the validation curve
plt.figure(figsize=(12, 6))

# Plot training and testing errors
plt.plot(degrees, train_mse, 'o-', linewidth=2, markersize=8, label='Training Error', color='#2ecc71')
plt.plot(degrees, test_mse, 's-', linewidth=2, markersize=8, label='Testing Error', color='#e74c3c')

# Find the optimal degree (minimum test error)
optimal_degree = degrees[np.argmin(test_mse)]
min_test_error = min(test_mse)

# Mark the optimal point
plt.axvline(x=optimal_degree, color='gray', linestyle='--', linewidth=1.5, alpha=0.7, label=f'Optimal Degree = {optimal_degree}')
plt.plot(optimal_degree, min_test_error, 'D', markersize=12, color='#f39c12', markeredgecolor='black', markeredgewidth=2, zorder=5)

# Add labels and formatting
plt.xlabel('Model Complexity (Polynomial Degree)', fontsize=14, fontweight='bold')
plt.ylabel('Mean Squared Error (MSE)', fontsize=14, fontweight='bold')
plt.title('Bias–Variance Tradeoff: Validation Curve', fontsize=16, fontweight='bold', pad=20)
plt.legend(fontsize=12, loc='upper right')
plt.grid(True, alpha=0.3)
plt.xticks(degrees)

# Add region labels
plt.text(1.5, max(test_mse) * 0.9, 'Underfitting\n(High Bias)', 
         fontsize=11, ha='center', color='#3498db', fontweight='bold',
         bbox=dict(boxstyle='round,pad=0.5', facecolor='white', edgecolor='#3498db', linewidth=2))

plt.text(optimal_degree, max(test_mse) * 0.75, 'Optimal\nComplexity', 
         fontsize=11, ha='center', color='#f39c12', fontweight='bold',
         bbox=dict(boxstyle='round,pad=0.5', facecolor='white', edgecolor='#f39c12', linewidth=2))

plt.text(9, max(test_mse) * 0.9, 'Overfitting\n(High Variance)', 
         fontsize=11, ha='center', color='#e74c3c', fontweight='bold',
         bbox=dict(boxstyle='round,pad=0.5', facecolor='white', edgecolor='#e74c3c', linewidth=2))

plt.tight_layout()
plt.show()

print(f"\n📊 Key Findings:")
print(f"  • Optimal polynomial degree: {optimal_degree}")
print(f"  • Minimum test error: {min_test_error:.4f}")

## Step 7: Additional Visualization - RMSE Comparison

In [ ]:
# Create RMSE plot for easier interpretation
plt.figure(figsize=(12, 6))

plt.plot(degrees, train_rmse, 'o-', linewidth=2, markersize=8, label='Training RMSE', color='#2ecc71')
plt.plot(degrees, test_rmse, 's-', linewidth=2, markersize=8, label='Testing RMSE', color='#e74c3c')

# Mark optimal point
optimal_test_rmse = test_rmse[optimal_degree - 1]
plt.axvline(x=optimal_degree, color='gray', linestyle='--', linewidth=1.5, alpha=0.7)
plt.plot(optimal_degree, optimal_test_rmse, 'D', markersize=12, color='#f39c12', markeredgecolor='black', markeredgewidth=2, zorder=5)

plt.xlabel('Model Complexity (Polynomial Degree)', fontsize=14, fontweight='bold')
plt.ylabel('Root Mean Squared Error (RMSE)', fontsize=14, fontweight='bold')
plt.title('RMSE vs Model Complexity', fontsize=16, fontweight='bold', pad=20)
plt.legend(fontsize=12, loc='upper right')
plt.grid(True, alpha=0.3)
plt.xticks(degrees)
plt.tight_layout()
plt.show()

print(f"\nRMSE is in the same units as the target (mg/m³), making it easier to interpret.")
print(f"Optimal model RMSE: {optimal_test_rmse:.4f} mg/m³")

## Step 8: Model Performance Comparison - R² Scores

In [ ]:
# Plot R² scores
plt.figure(figsize=(12, 6))

plt.plot(degrees, train_r2, 'o-', linewidth=2, markersize=8, label='Training R²', color='#2ecc71')
plt.plot(degrees, test_r2, 's-', linewidth=2, markersize=8, label='Testing R²', color='#e74c3c')

# Mark optimal point
plt.axvline(x=optimal_degree, color='gray', linestyle='--', linewidth=1.5, alpha=0.7)
plt.axhline(y=1.0, color='black', linestyle=':', linewidth=1, alpha=0.5, label='Perfect Fit (R²=1)')

plt.xlabel('Model Complexity (Polynomial Degree)', fontsize=14, fontweight='bold')
plt.ylabel('R² Score (Coefficient of Determination)', fontsize=14, fontweight='bold')
plt.title('Model Performance: R² Score vs Complexity', fontsize=16, fontweight='bold', pad=20)
plt.legend(fontsize=12, loc='lower right')
plt.grid(True, alpha=0.3)
plt.xticks(degrees)
plt.ylim([0, 1.1])
plt.tight_layout()
plt.show()

print(f"\nR² scores show how much variance in CO concentration is explained by the model.")
print(f"Optimal model R² (test): {test_r2[optimal_degree - 1]:.4f}")

## Step 9: Detailed Analysis - Gap Between Training and Testing Errors

In [ ]:
# Calculate the gap between training and testing errors
error_gap = np.array(test_mse) - np.array(train_mse)

plt.figure(figsize=(12, 6))
plt.plot(degrees, error_gap, 'o-', linewidth=2.5, markersize=10, color='#9b59b6')
plt.axhline(y=0, color='black', linestyle='-', linewidth=1)
plt.axvline(x=optimal_degree, color='gray', linestyle='--', linewidth=1.5, alpha=0.7)

plt.xlabel('Model Complexity (Polynomial Degree)', fontsize=14, fontweight='bold')
plt.ylabel('Error Gap (Test MSE - Train MSE)', fontsize=14, fontweight='bold')
plt.title('Overfitting Indicator: Gap Between Test and Train Errors', fontsize=16, fontweight='bold', pad=20)
plt.grid(True, alpha=0.3)
plt.xticks(degrees)

# Highlight regions
plt.fill_between(degrees, 0, error_gap, where=(np.array(error_gap) > 0), 
                 alpha=0.3, color='red', label='Overfitting Region')

plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

print("\nA larger gap indicates more overfitting (model memorizing training data).")
print("\nError gaps by degree:")
for d, gap in zip(degrees, error_gap):
    print(f"  Degree {d}: {gap:.4f}")

---

## Discussion Questions

### Question 1: Which polynomial degree gives the best generalization?

**Answer:**

Based on our validation curve analysis, the polynomial degree that gives the best generalization is **degree {optimal_degree}**. This is determined by finding the degree that yields the **minimum testing error**.

**Key observations:**
- At this degree, the model achieves a good balance between bias and variance
- The testing error is at its minimum, indicating the best performance on unseen data
- The gap between training and testing errors is relatively small, suggesting the model is not overfitting significantly
- This degree captures the essential patterns in the relationship between meteorological variables and CO concentration without fitting noise

**Why this matters:**
- A model that generalizes well will perform reliably on new, unseen air quality measurements
- For practical applications in environmental monitoring, we want predictions that are accurate for future observations, not just our historical data

### Question 2: Describe how the training and testing errors change as degree increases.

**Answer:**

The behavior of errors as polynomial degree increases follows a characteristic pattern that illustrates the bias-variance tradeoff:

**Training Error:**
- **Monotonically decreases** as degree increases
- At degree 1 (linear model): relatively high due to underfitting
- As degree increases: the model becomes more flexible and fits the training data better
- At high degrees (8-10): approaches very low values as the model can fit almost every point
- This continuous decrease shows the model's increasing capacity to fit the training data

**Testing Error:**
- Follows a **U-shaped curve** (the hallmark of the bias-variance tradeoff)
- **Low degrees (1-2):** High testing error due to underfitting (model too simple)
- **Medium degrees (3-5):** Testing error reaches its minimum (optimal complexity)
- **High degrees (6-10):** Testing error increases due to overfitting (model too complex)

**The Gap:**
- At low degrees: small gap (model performs similarly on both sets, but poorly)
- At optimal degree: moderate gap (acceptable generalization)
- At high degrees: large and growing gap (severe overfitting)

This divergence between training and testing errors at high complexity is the signature of overfitting.

### Question 3: Explain how bias and variance manifest in this dataset.

**Answer:**

**Bias (Underfitting) - Low Complexity Models:**

Bias manifests in our low-degree polynomial models (degrees 1-2):
- The linear model (degree 1) makes strong assumptions about the relationship between features and CO concentration
- **Evidence:** Both training and testing errors are high
- **Cause:** The model cannot capture the true complexity of how temperature, humidity, and CO concentration interact
- **Result:** Systematic errors - the model consistently misses important patterns
- **Real-world interpretation:** Using only a linear relationship is too simplistic for atmospheric chemistry, where interactions may be nonlinear

**Variance (Overfitting) - High Complexity Models:**

Variance manifests in our high-degree polynomial models (degrees 7-10):
- The model has so many parameters that it fits noise in the training data
- **Evidence:** Training error very low, but testing error high (large gap)
- **Cause:** The model is too sensitive to small fluctuations in the training data
- **Result:** Random errors - the model performs very differently on training vs. testing data
- **Real-world interpretation:** The model memorizes sensor noise, measurement errors, and random variations specific to the training period, rather than learning true atmospheric patterns

**The Tradeoff:**
- As we increase model complexity, bias decreases (model can fit more complex patterns)
- But variance increases (model becomes more sensitive to training data specifics)
- The optimal degree represents the best balance where total error (bias² + variance) is minimized

### Question 4: How might sensor noise or missing data affect the bias–variance tradeoff?

**Answer:**

Sensor noise and missing data have significant impacts on the bias-variance tradeoff:

**Impact of Sensor Noise:**

1. **Increases Variance:**
   - Noise in sensor readings adds randomness to the training data
   - Complex models (high degree polynomials) will try to fit this noise
   - This shifts the optimal complexity toward simpler models
   - The gap between training and testing errors becomes larger at lower degrees

2. **Overfitting Occurs Earlier:**
   - With noisy data, even moderately complex models may start overfitting
   - The optimal degree might be lower (e.g., degree 2-3 instead of 4-5)
   - High-degree models would perform especially poorly

3. **Real-world implications:**
   - Air quality sensors have inherent measurement uncertainties
   - Drift in sensor calibration over time adds noise
   - Environmental factors (temperature, humidity) can affect sensor accuracy

**Impact of Missing Data:**

1. **Reduces Sample Size:**
   - Fewer training examples means less information to learn from
   - Complex models need more data; with less data, they're more likely to overfit
   - This also shifts optimal complexity toward simpler models

2. **May Introduce Bias:**
   - If data is missing non-randomly (e.g., sensors fail during extreme conditions)
   - The model won't learn about those conditions
   - This creates systematic bias in predictions for those scenarios

3. **Affects Generalization:**
   - Less representative training data → worse generalization
   - Testing error may be higher overall
   - The model might not encounter important patterns during training

**Practical Strategies:**
- Use simpler models when data is noisy or sparse
- Apply regularization techniques (Ridge, Lasso) to reduce overfitting
- Consider data quality before choosing model complexity
- Use cross-validation to get more robust estimates of generalization error
- Investigate why data is missing and whether it introduces systematic bias

---

## Bonus: Cross-Validation Analysis (Optional)

Instead of a single train-test split, we can use k-fold cross-validation to get a more robust estimate of model performance.

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform 5-fold cross-validation for each degree
cv_scores = []
cv_std = []

print("Performing 5-fold cross-validation...\n")
print(f"{'Degree':<8} {'CV MSE Mean':<15} {'CV MSE Std':<15}")
print("-" * 40)

for degree in degrees:
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly.fit_transform(X)
    
    # Create and evaluate model with cross-validation
    model = LinearRegression()
    
    # Note: cross_val_score with scoring='neg_mean_squared_error' returns negative MSE
    scores = cross_val_score(model, X_poly, y, cv=5, scoring='neg_mean_squared_error')
    
    # Convert to positive MSE
    mse_scores = -scores
    
    cv_scores.append(mse_scores.mean())
    cv_std.append(mse_scores.std())
    
    print(f"{degree:<8} {mse_scores.mean():<15.4f} {mse_scores.std():<15.4f}")

print("\nCross-validation complete!")

In [ ]:
# Compare single split vs. cross-validation
plt.figure(figsize=(14, 6))

# Plot single split results
plt.plot(degrees, test_mse, 's-', linewidth=2, markersize=8, label='Single Split (30% Test)', color='#e74c3c')

# Plot cross-validation results with error bars
plt.errorbar(degrees, cv_scores, yerr=cv_std, fmt='o-', linewidth=2, markersize=8, 
             capsize=5, capthick=2, label='5-Fold Cross-Validation', color='#3498db')

# Find optimal degrees for both methods
optimal_single = degrees[np.argmin(test_mse)]
optimal_cv = degrees[np.argmin(cv_scores)]

plt.axvline(x=optimal_single, color='#e74c3c', linestyle='--', linewidth=1.5, alpha=0.5, label=f'Optimal (Single): {optimal_single}')
plt.axvline(x=optimal_cv, color='#3498db', linestyle='--', linewidth=1.5, alpha=0.5, label=f'Optimal (CV): {optimal_cv}')

plt.xlabel('Model Complexity (Polynomial Degree)', fontsize=14, fontweight='bold')
plt.ylabel('Mean Squared Error (MSE)', fontsize=14, fontweight='bold')
plt.title('Comparison: Single Split vs Cross-Validation', fontsize=16, fontweight='bold', pad=20)
plt.legend(fontsize=11, loc='upper right')
plt.grid(True, alpha=0.3)
plt.xticks(degrees)
plt.tight_layout()
plt.show()

print(f"\n📊 Comparison:")
print(f"  • Optimal degree (single split): {optimal_single}")
print(f"  • Optimal degree (cross-validation): {optimal_cv}")
print(f"\n  Cross-validation provides more reliable estimates by testing on multiple different data splits.")
print(f"  The error bars show the variability in performance across different folds.")

---

## Summary and Conclusions

### Key Takeaways:

1. **Bias-Variance Tradeoff is Real:**
   - Simple models (low degree) suffer from high bias → underfitting
   - Complex models (high degree) suffer from high variance → overfitting
   - The optimal model balances both

2. **Model Selection Matters:**
   - Always evaluate on held-out test data
   - Training error alone is misleading
   - Cross-validation provides more robust estimates

3. **Real-World Applications:**
   - For air quality prediction, moderate complexity works best
   - Consider data quality (noise, missing values) when choosing complexity
   - Simpler models are often more interpretable and reliable

4. **Engineering Implications:**
   - Understanding this tradeoff helps in:
     - Designing sensor networks
     - Building predictive maintenance systems
     - Creating environmental monitoring tools
     - Developing early warning systems

### What We Learned:

- How to implement polynomial regression in Python
- How to evaluate model performance using MSE, RMSE, and R²
- How to visualize the bias-variance tradeoff
- How to interpret validation curves
- The importance of testing on unseen data
- How cross-validation provides better estimates

---

**Lab completed successfully! 🎉**